In [1]:
# I'm not going to write a web application... because that's dumb.  But I'll do something similar.

from hashlib import sha1
from random import choices
from time import sleep, time

SECRET_KEY = b'\xe7\xef\x1e\x7f\xd7\x87\xa4\xeb\x10<\xd9\x9f\x8b\xec\x03\x8f'
DELAY = 0.001 #I'm doing 100ms instead of 50 just for convenience

def Validate(data: bytes, potential_hmac: bytes):
    hmac = sha1(SECRET_KEY+data).digest()
    assert len(potential_hmac) == len(hmac)
    for n in range(len(hmac)):
        sleep(DELAY)
        if hmac[n] != potential_hmac[n]:
            return False
    
    return True

# Use this just for debugging
def HMAC(data: bytes):
    return list(sha1(SECRET_KEY+data).digest())

In [2]:
# Make a random "file"

data = bytes(choices(range(256), k = 500))
print(data)

b'\xf7\xf32\xca\nK\xcf\x81\xf4\x19\x0c\xbe\x87\xbd\xa5Dt}\xbb\xaa\xb7\xdd\x1b\x1e\x81\x0b\x98\xea\xf6\x87\x14\xf6\xbe\xda5<\xd0\x8b6lN\x83\x19<\xf9\x12\x8b+\xba\xf7\x04\xb1\xf2\xb2\x84w\x8f\xfb\xdeC\x02\x15\xd2r\x7f\xa03\x825}\xc2\xaa\x18\xfe\xc9K8\x13\x82\xc7\r\xdf%\x12\xd7\x8bXbB\xfc\x1eP\x0fNQ\xb2\x80W\xd0R\xea\x10\xa4\n\xe9\x14\x8f3\xe5\x80\xdc\x12\xe9\xe0mM\xc2\xbf\xf6.I\xfcvn\xa1\xc2\xb7\xc8Z\xfb\xf9S|\xb6L\xb0GM\x92,\xc1BM\x97I\xb3\x8e\x90\x80#\x92\xd4\xad\xe8\xf4;\niN.\xbb(\x1c\xb7\x8fv\x0fu`\x0b\x07\xf4\x14\xa8X\x06 =t\x07\xecf\xeb\x8cI\x89\xfa\xe0\x0b\x99\x06_\x05\xf0\xc2\t\xad\xa8\xe3\xdcb\xcc:I%\x93\x9a\xef\x14\xef\xfc\xc2_\x1d\xe1\xeb\xceV0\xe3\xd4\x98\x10\xaa\xe3\x02J\t\xf3~]\xc9\x08fS\xc0\xfa\xc0\xb1\xaf\x06{\xb6\xcd\xf6\x10\xae%\x00~\x17\xac\x1d/\t)\xcc\xd7\xd8\xee\xaar_x\xf8\x08lI\xe1\r\x95k@\x9e\xde\n/\xee\x9b\x9b\xcb+\x18\xb7\x07\xfc\xc3H\xe6\xa9rX\xef\xbf\x8c\xa5J\xf9\xde\xf3(J\xc1~\xda\xda*\n\xd2\xc0\xc8%4aAb\xe2\xc9&\x7f}\xe2[$\xb1\x9b_\x9a\x08\xc2Q\xdf#\xb8i\xa5\

In [3]:
known_bytes = []
TOTAL_HMAC_BYTES = 20
response = False

# The way I'm going to handle this is test each character 10 times and choose the one that had the total longest response.
# Taking multiple samples should help account for any random fluctuations in the timing, which I encountered previously.

for n in range(TOTAL_HMAC_BYTES):
    if response == True:
        break
    print('Known bytes:', known_bytes)
    dt = [0] * 256
    for char in tuple(range(256))*10:
        guess = [ *known_bytes, char ] + [0] * ( TOTAL_HMAC_BYTES - len(known_bytes) - 1 )
        tic = time()
        response = Validate(data, guess)
        toc = time()
        if response == True:
            # I guess we're done
            known_bytes += [char]
            break
        else:
            dt[char] += toc-tic
    
    if response == False:
        max_dt = max(dt)
        best_char = dt.index(max_dt)
        known_bytes += [best_char]

            
    

Known bytes: []
Known bytes: [40]
Known bytes: [40, 71]
Known bytes: [40, 71, 127]
Known bytes: [40, 71, 127, 20]
Known bytes: [40, 71, 127, 20, 48]
Known bytes: [40, 71, 127, 20, 48, 43]
Known bytes: [40, 71, 127, 20, 48, 43, 154]
Known bytes: [40, 71, 127, 20, 48, 43, 154, 80]
Known bytes: [40, 71, 127, 20, 48, 43, 154, 80, 84]
Known bytes: [40, 71, 127, 20, 48, 43, 154, 80, 84, 59]
Known bytes: [40, 71, 127, 20, 48, 43, 154, 80, 84, 59, 157]
Known bytes: [40, 71, 127, 20, 48, 43, 154, 80, 84, 59, 157, 211]
Known bytes: [40, 71, 127, 20, 48, 43, 154, 80, 84, 59, 157, 211, 96]
Known bytes: [40, 71, 127, 20, 48, 43, 154, 80, 84, 59, 157, 211, 96, 254]
Known bytes: [40, 71, 127, 20, 48, 43, 154, 80, 84, 59, 157, 211, 96, 254, 249]
Known bytes: [40, 71, 127, 20, 48, 43, 154, 80, 84, 59, 157, 211, 96, 254, 249, 104]
Known bytes: [40, 71, 127, 20, 48, 43, 154, 80, 84, 59, 157, 211, 96, 254, 249, 104, 178]
Known bytes: [40, 71, 127, 20, 48, 43, 154, 80, 84, 59, 157, 211, 96, 254, 249, 104, 

In [4]:
print(known_bytes)

[40, 71, 127, 20, 48, 43, 154, 80, 84, 59, 157, 211, 96, 254, 249, 104, 178, 202, 50, 211]


In [5]:
Validate(data, known_bytes)

True